In [22]:
#%pip install python-docx

In [1]:
from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter
import io
import os
import textract
import docx

In [2]:
projectDir = '/Users/shishen/Insight_Data_Science/Resume_Standardization/'
resumeDir = projectDir + 'data/raw_resume/'

In [3]:
def convert_pdf_to_txt(path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
    page_interpreter = PDFPageInterpreter(resource_manager, converter)

    with open(path, 'rb') as fh:

        for page in PDFPage.get_pages(fh,
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)

        text = fake_file_handle.getvalue()

    # close open handles
    converter.close()
    fake_file_handle.close()

    return text

In [4]:
resumeTxt = []
for filename in os.listdir(resumeDir):
    if(filename.endswith(".pdf")):
        try:
            resumeTxt.append(convert_pdf_to_txt(resumeDir+filename))
        except Exception:
            print('Error reading .pdf file' + filename)
    if(filename.endswith(".docx")):
        try:
            doc = docx.Document(resumeDir + filename)
            data = ""
            fullText = []
            for para in doc.paragraphs:
                fullText.append(para.text)
                data = '\n'.join(fullText)
            resumeTxt.append(data)
        except IOError:
            print('Error reading .docx file' + filename)
    if(filename.endswith(".doc")):
        try:
            text = textract.process(resumeDir+filename).decode('utf-8')     
            resumeTxt.append(text)
        except Exception:
            print('Error reading .doc file :' + filename)

In [6]:
txt_file = projectDir + 'data/clean_resume/resume'

In [11]:
i = 0
for resume in resumeTxt:
    file = open(txt_file + str(i) + '.txt', 'w+')
    file.writelines(resume)
    file.close()
    i += 1